# Generating WISE entries

## Adding a new dababase field

Firstly, ensure that you have defined a custom field in `config.ini`. See [official documentation](https://arkime.com/settings#custom-fields) for more info.

```ini
[custom-fields]
target.name=kind:lotermfield;count:true;friendly:Name;db:target.name;help:Target name
```

Note that this can be done in multiple ways. It can be defined within the WISE data file itself. Or it can be added in WISE configuration. I personally prefer arkime main config as central source of truth. 

Also, don't forget to add this into session view.

```ini
[custom-views]
cdmcs=title:Cyber Defence Monitoring Course;require:target;fields:target.name
```

Finally, make sure WISE is configured to load the targets TXT file.

```
[file:ip]
file=/vagrant/targets.txt
tags=ipwise
type=ip
format=tagger
```

## Define the arkime txt header

Firstly, we define a field shortcut for a field that has already been created in arkime main config.

In [1]:
FIELDS = [
    "#field:target.name;shortcut:0",
]

We can also create new fields in the database file itself. Though different options in field creation can easily lead to framented and confusing configuration. But since we're generating the databse file in code anyway, we might as well make the field generation more structured. Making field creation lines manually is very much prone to errors.

In [2]:
NEW_FIELD = [
    "#field:whatever.int",
    "kind:integer",
    "count:false",
    "friendly:A Integer",
    "db:whatever.dbint",
    "help:Help for integer",
    "shortcut:1"
]

In [3]:
FIELDS.append(";".join(NEW_FIELD))

Then we create a structured definition of fields. This is a simple example. In reality, this data structure would be parsed from somewhere.

In [4]:
DATA_IP = [
    {
        "ip": "192.168.56.11",
        "name": "vagrant_host",
        "whatever": 13
    },
    {
        "ip": "8.8.8.8",
        "name": "google_dns",
        "whatever": 666
    },
    {
        "ip": "10.0.2.15",
        "name": "vagrant",
        "whatever": 0,
    }
]

In [5]:
TEMPLATE = "{ip};0={name};1={whatever}"

`format_map`on python format string simply replaces a named placeholder with corresponding dictionary key. This is equal to `TEMPLATE.format(ip=item["ip"], name=item["name"]), whatever=item["whatever"]`.

In [6]:
for item in DATA_IP:
    FIELDS.append(TEMPLATE.format_map(item))

In [7]:
FIELDS

['#field:target.name;shortcut:0',
 '#field:whatever.int;kind:integer;count:false;friendly:A Integer;db:whatever.dbint;help:Help for integer;shortcut:1',
 '192.168.56.11;0=vagrant_host;1=13',
 '8.8.8.8;0=google_dns;1=666',
 '10.0.2.15;0=vagrant;1=0']

Generated lines can then be concatenated with newline to create database file content.

In [8]:
FIELDS = "\n".join(FIELDS)
print(FIELDS)

#field:target.name;shortcut:0
#field:whatever.int;kind:integer;count:false;friendly:A Integer;db:whatever.dbint;help:Help for integer;shortcut:1
192.168.56.11;0=vagrant_host;1=13
8.8.8.8;0=google_dns;1=666
10.0.2.15;0=vagrant;1=0


And finally we dump the result into a file handle.

In [9]:
with open("targets.txt", "w") as handle:
    handle.write(FIELDS)